In [1]:
import os
import sys
import torch

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
from graph_bridges.configs.graphs.config_sb import SBConfig
from graph_bridges.configs.graphs.config_sb import TrainerConfig, ParametrizedSamplerConfig
from graph_bridges.data.graph_dataloaders_config import EgoConfig,CommunityConfig,CommunitySmallConfig
from graph_bridges.models.backward_rates.backward_rate_config import BackRateMLPConfig
from graph_bridges.configs.graphs.config_sb import SBConfig, ParametrizedSamplerConfig, SteinSpinEstimatorConfig
from graph_bridges.models.backward_rates.backward_rate_config import GaussianTargetRateImageX0PredEMAConfig

In [3]:
from graph_bridges.models.generative_models.sb import SB
from graph_bridges.models.metrics.sb_metrics import paths_marginal_histograms

In [12]:
config = SBConfig(delete=True)

#config.data = EgoConfig(as_image=False, batch_size=32, full_adjacency=False)
config.data = CommunityConfig(as_image=False, batch_size=32, full_adjacency=False)
#config.data = EgoConfig(as_image=False, batch_size=32, full_adjacency=False)
#config.data = CommunitySmallConfig(as_image=False, batch_size=32, full_adjacency=False)
config.model = GaussianTargetRateImageX0PredEMAConfig(time_embed_dim=12, fix_logistic=False)

#config.model = BackRateMLPConfig(time_embed_dim=14,hidden_layer=150)
config.stein = SteinSpinEstimatorConfig(stein_sample_size=100)
config.sampler = ParametrizedSamplerConfig(num_steps=10,step_type="Poisson")
config.optimizer = TrainerConfig(learning_rate=1e-3,
                                 num_epochs=200,
                                 save_metric_epochs=20,
                                 metrics=["graphs_plots",
                                          "histograms"])
config.align_configurations()
#read the model
device = torch.device("cpu")
sb = SB(config, device)

In [13]:
databatch = next(sb.data_dataloader.train().__iter__())
x_spins_data = databatch[0]

In [14]:
print("From Dataloader full path in image shape with times")
spins_path_1, times_1 = sb.pipeline(None, 0, device, return_path=True,return_path_shape=True)

print("From given start")
spins_path_2, times_2 = sb.pipeline(sb.training_model,1,device,x_spins_data,return_path=True,return_path_shape=True)

From Dataloader full path in image shape with times


10it [00:00, 1540.27it/s]


From given start


10it [00:00, 155.57it/s]


In [17]:
backward_histogram,forward_histogram,forward_time = paths_marginal_histograms(sb=sb,
                                                                              sinkhorn_iteration=0,
                                                                              device=device,
                                                                              current_model=sb.training_model,
                                                                              past_to_train_model=None)

10it [00:00, 1396.01it/s]
10it [00:00, 157.66it/s]
10it [00:00, 979.31it/s]
10it [00:00, 185.55it/s]
10it [00:00, 1081.90it/s]
10it [00:00, 138.18it/s]
10it [00:00, 1280.55it/s]
10it [00:00, 117.76it/s]
10it [00:00, 1250.84it/s]
10it [00:00, 134.69it/s]
10it [00:00, 1153.26it/s]
10it [00:00, 134.96it/s]
10it [00:00, 1352.35it/s]
10it [00:00, 129.73it/s]
10it [00:00, 1633.68it/s]
10it [00:00, 134.65it/s]
10it [00:00, 1555.69it/s]
10it [00:00, 130.01it/s]
10it [00:00, 1331.65it/s]
10it [00:00, 143.74it/s]
10it [00:00, 1524.04it/s]
10it [00:00, 146.06it/s]
10it [00:00, 1377.89it/s]
10it [00:00, 119.59it/s]
10it [00:00, 1045.70it/s]
10it [00:00, 125.42it/s]
10it [00:00, 1534.69it/s]
10it [00:00, 114.64it/s]
10it [00:00, 986.62it/s]
10it [00:00, 132.29it/s]
10it [00:00, 1305.90it/s]
10it [00:00, 124.14it/s]
10it [00:00, 910.28it/s]
10it [00:00, 60.02it/s]
10it [00:00, 1177.84it/s]
10it [00:00, 90.73it/s]
10it [00:00, 924.77it/s]
10it [00:00, 84.93it/s]
10it [00:00, 889.81it/s]
10it [00:00, 

In [18]:
from graph_bridges.models.metrics.data_metrics import SpinBernoulliMarginal

marginal_0 = SpinBernoulliMarginal(spin_dataloader=sb.data_dataloader)()
marginal_1 = SpinBernoulliMarginal(spin_dataloader=sb.target_dataloader)()

5
25
